In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import keras_preprocessing

train_dir = '/content/drive/MyDrive/Dataset Fix'


train_datagen = keras_preprocessing.image.ImageDataGenerator(
      rescale=1./255,
      validation_split = 0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    subset='training')

# Create a validation generator
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    subset='validation')

Found 20 images belonging to 4 classes.
Found 4 images belonging to 4 classes.


In [ ]:
pre_trained_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
pre_trained_model.trainable = False

model = tf.keras.Sequential([
    pre_trained_model,  
    tf.keras.layers.Conv2D(32, 3, activation='relu'),  
    tf.keras.layers.Dropout(0.2),  
    tf.keras.layers.GlobalAveragePooling2D(),  
    tf.keras.layers.Dense(4, activation='softmax')  
])


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy']) 

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 4)                 132       
Total params: 2,626,788
Trainable params: 368,804
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
history = model.fit(train_generator, epochs=20, validation_data=val_generator)

Epoch 1/20
1/1 [==============================] - 6s 6s/step - loss: 2.4062 - accuracy: 0.2000 - val_loss: 2.9648 - val_accuracy: 0.7500
Epoch 2/20
1/1 [==============================] - 1s 609ms/step - loss: 2.4822 - accuracy: 0.6500 - val_loss: 2.4389 - val_accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 1s 586ms/step - loss: 1.8731 - accuracy: 0.7500 - val_loss: 1.4253 - val_accuracy: 0.7500
Epoch 4/20
1/1 [==============================] - 1s 602ms/step - loss: 0.7984 - accuracy: 0.8000 - val_loss: 0.2828 - val_accuracy: 0.7500
Epoch 5/20
1/1 [==============================] - 1s 583ms/step - loss: 0.1245 - accuracy: 0.9500 - val_loss: 0.0320 - val_accuracy: 1.0000
Epoch 6/20
1/1 [==============================] - 1s 588ms/step - loss: 0.0432 - accuracy: 1.0000 - val_loss: 0.0344 - val_accuracy: 1.0000
Epoch 7/20
1/1 [==============================] - 1s 589ms/step - loss: 0.0555 - accuracy: 1.0000 - val_loss: 0.0488 - val_accuracy: 1.0000
Epoch 8/20
1/1 [=======

In [ ]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 13.0MB/s 


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
from extracted_face import extract_face 

path = '/content/drive/MyDrive/Face Dataset/Validation/C1151466/download (8).jpg'
img = extract_face(path)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
predictions = model.predict(images/255.0)
score = tf.nn.softmax(predictions[0])
print(predictions[0])

IndexError: ignored

In [ ]:
model.save('/content/my_model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
